In [ ]:
# Step 1: Load relevant sections from DeepSeek tagging, filter to desired dates (make sure to remove NaT and invalid dates)
# Step 2: Analyze each section using problem solution prompt with ChatGPT 4.1
# Step 3: Load ChatGPT response as JSON

In [2]:
# libraries needed for this file 
from openai import OpenAI
import json
import os
from ast import literal_eval
import pandas as pd
import re
from tabulate import tabulate

#-------------------------------------------------------------

# set vars here
TOPIC = "school closures" # the topic to be mined
init_deepseek_results = 'deepseek_kw_chunks_2025.csv' # the deepseek CSV relevancy results w/o dates (skip this in future if file has dates)
DEEPSEEK_RESULTS = "deepseek_results_24_25_only.csv" # where to save the deepseek CSV relevancy results with dates
GPT_RESULTS_DIR = 'PROBLEM_SOLUTION_GPT41_JSON_RESULTS' #

# this is our lab key to use ChatGPT
KEY = "" # YOUR KEY HERE :D
client = OpenAI(api_key=KEY)

#-------------------------------------------------------------
# FUNCTIONS 

def truncate_section(section, max_tokens, verbose = False) :
    '''
    Params: 
    - section (str): text from relevant transcript section from deepseek file
    - max_tokens (int): token limit for llm
    - verbose (boolean): show print statements or not

    Returns: str
    
    Description:
    - crops section to max tokens by truncating centrally
    - used in problem_solution_prompt'''
    
    start = (len(section) - max_tokens) // 2
    end = start + max_tokens
    truncated_result = section[start:end]
    
    if verbose:
        print('--truncated section--')
        
    return truncated_result


def problem_solution_prompt(transcript,this_video_id,max_tokens):
    '''
    Params: 
    - transcript (str): text from relevant transcript section from deepseek file
    - this_video_id (str): unique video id from district view associated with this text
    - max_tokens (int): token limit for llm
    
    Returns: str
    
    Description:
    - defines problem solution prompt with instructions, example output, & required JSON output
    - appends section text to instruction for final prompt construction
    - ensures final prompt with text is within length of max tokens by truncating 
    text centrally and only as needed to avoid cropping prompt
    '''

    prompt_instructions = f"""
You are an advanced transcript topic analyzer specializing in extracting structured problem solution insights from school board meeting transcripts discussing TOPIC. 
Your task is to write a problem solution analysis of the input denoted by TRANSCRIPT at the end of this prompt related to {TOPIC}. 
You will return your analysis in a strict JSON format following the JSON OUTPUT STRUCTURE and adhering to the following INSTRUCTIONS.

### Instructions:
    1. Always return a valid JSON object precisely following the JSON OUTPUT STRUCTURE.
    2. Do not include any text, explanations, or preambles outside the JSON structure.
    3. EXTREMELY IMPORTANT: Do not wrap your response in ```json or ``` markdown code blocks.
    4. If a transcript does not contain relevant information, return an empty value (`null`, `[]`, or `""` where appropriate).  
    5. Phrase problems as clearly defined challenges, and solutions as actionable steps or strategies.
    6. CRITICAL: Ensure each solution is only included under the problem keys it directly addresses. Do not generalize or duplicate solutions across unrelated problems.

### VIDEO_ID: {this_video_id}

### JSON OUTPUT STRUCTURE:
{{
  "video_id": VIDEO_ID,
  "problem_solution_dictionary": {{
      "<problem_statement_1>": ["<solution_1>", "<solution_2>", ...], "<problem_statement_2>": [] }} 
}}

### EXAMPLE OUTPUT:
{{
  "video_id": "123abcDEF!!",
  "problem_solution_dictionary": {{ 
      "Lack of communication transparency in closure decision affecting trust of parents and community members": ["Provide platforms for public to view data dashboards","Send out community communications expectation survey to gather feedback"],
      "Special education resources may be affected by closure with staff being stretched thin across larger classes": ["Ensure special education budgeting is updated to accommodate for larger class sizes"],
      "Parents struggling with transportation due to unequal student distribution after school closure": [] }}
}}



### TRANSCRIPT:
"""
    
    prompt = f'{prompt_instructions}\n{transcript}'

    current_prompt_length = len(prompt)

    if current_prompt_length >= max_tokens:

        # truncate but dont cut instructions
        instruction_length = len(prompt_instructions)

        # getting max length transcript should be based on max prompt length
        max_tokens_transcript = max_tokens - instruction_length - 1 #account for new line

        truncated_result = truncate_section(section=transcript,max_tokens=max_tokens_transcript)

        #update the prompt
        prompt = f'{prompt_instructions}\n{truncated_result}'

    return prompt



def analyze_transcript_sections(transcript_sections,this_video_id,prompt_func, model="gpt-4.1"): #model="gpt-4o"
    '''
    Params: 
    - transcript_sections (list of str): list of text sections representing relevant parts of
    transcripts related to the topic
    - this_video_id (str): unique video id from district view associated with this text
    - prompt_func (function): which prompt function to be used
    - model(str): llm to be used, default model is gpt-4.1
    
    Returns: list of JSON
    
    Description:
    - for each of the relevant transcript sections in a transcript, 
    annote section using the llm model & specified prompt
    - Outputs list of jsons corresponding to each section
    '''
    

    results = []

    for transcript in transcript_sections:

        prompt = prompt_func(transcript = transcript,this_video_id=this_video_id,max_tokens=8000)

        prompt_tokens = len(prompt)
        if prompt_tokens <= 8000: # model limit is 8100

            response = client.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]

            )

            try:
                response_content = response.choices[0].message.content

                result_json = json.loads(response_content)


            except json.JSONDecodeError:
                print("Failed to parse JSON:", response_content)
                result_json = {"error": f"Invalid JSON response - RAW RESPONSE: **{response_content}**"}

            results.append(result_json)

        else:
            result_json = {"error": f"Too many tokens = {prompt_tokens}"}
            results.append(result_json)


    return results



    
def get_llm_annotated_jsons(file_lst,save_directory =  GPT_RESULTS_DIR):
    '''
    Params: 
    - file_lst (list of str): list of files containing sections to be annotated
    - save_directory (str): directory to save results, default to GPT_RESULTS_DIR

    Returns: None
    
    Description:
    - iterates through given csv file list of topic related meetings
    - finds sections tagged as relevant from deepseek, annotes with problem solution prompt
    - saves raw llm json output results in specified directory
    '''

    os.makedirs(save_directory, exist_ok=True)

    # checking where I am
    print(os.getcwd())

    for input_filename in file_lst:
        index = file_lst.index(input_filename) + 1
        print(index, ' / ', len(file_lst))

        deepseek_df = pd.read_csv(input_filename)
        deepseek_df = deepseek_df.drop_duplicates(subset=['video_id'])

        print(deepseek_df.columns)

        # get the kept sections
        transcripts= deepseek_df['kept_sections'].apply(literal_eval)
        print("total number of transcript sections to be annotated", transcripts.explode().count())

        vid_id_lst = deepseek_df['video_id'].tolist()
        print('Number of unique video ids in this file', len(vid_id_lst))

        all_transcripts_for_this_file = []

        for t_list, video_id in zip(transcripts, vid_id_lst):
            output = analyze_transcript_sections(transcript_sections=t_list, this_video_id=video_id,prompt_func=problem_solution_prompt, model="gpt-4.1")
            all_transcripts_for_this_file.append(output)

        json_filename = f'GPT4o_output_{input_filename}.json'
        save_file_path = os.path.join(save_directory, json_filename)

        # save gpt4 output
        with open(save_file_path, "w") as file:
            json.dump(all_transcripts_for_this_file, file, indent=4)

        # show in terminal
        print(f"File saved at: {save_file_path}")

        # show in terminal
        print(json.dumps(all_transcripts_for_this_file, indent=2))

    print('Completed gpt4o tagging for all files.')


    
def check_keys(json_item, required_keys):
    '''
    Params: 
    - json_item: json / dict
    - required_keys: list of keys to check for 
    
    Returns: boolean
    
    Description:
    - returns true iff all keys and subkeys are present
    '''

    key_available_bool = all(key in json_item for key in required_keys)
    return key_available_bool


    
def custom_json_loads(json_file):
    '''
    Params: 
    - json_file (str): json file path
    
    Returns: json
    
    Description:
    - loads data from file path and returns validated json sections
    '''

    required_keys = {"video_id","problem_solution_dictionary"}

    valid_data = []

    with open(json_file, 'r') as file:
        data = json.load(file)

        for section_list in data:

            # may have multiple sections so need to save each individually for
            for section in section_list:
                has_keys_bool = check_keys(section,required_keys)

                # if response is good to go, save section as row
                if has_keys_bool == True:
                    valid_section = {} # new dictionary representing a single section

                    for key in required_keys:
                        valid_section[key] = section[key]

                    valid_data.append(valid_section)
                else:
                    continue # just skip invalid sections

    return valid_data



def run_gpt_annotation_for_file(files = [DEEPSEEK_RESULTS]):
    '''
    Params: 
    - files (list of str): list of deepseek relevancy files, 
    defaulted to contain single file [DEEPSEEK_RESULTS]
    
    Returns: str
    
    Description: 
    - runs chatgpt problem solution annotation for each topic relevant section for each file
    - prints resulting json filenames after llm tagging is completed
    '''

    kept_csv_lst = files

    # iterate through each file, get gpt4o output
    get_llm_annotated_jsons(kept_csv_lst) 

    print('----moving onto validating and loading jsons as dataframes ----')


    all_filenames = os.listdir(GPT_RESULTS_DIR)

    raw_json_filenames = []
    for file in all_filenames:
        if (any(kept in file for kept in kept_csv_lst)) and file.startswith("GPT4o_output") and file.endswith(".json") and 'example' not in file:
            raw_json_filenames.append(file)

    print(raw_json_filenames)  # show the resulting filenames





In [3]:
# !SKIP THIS STEP IN THE FUTURE by ensuring district view data includes meeting_date col!


# merge meeting date file to get dates

dates = pd.read_json('../districtview2025_with_dates.topics.jsonl', lines = True).drop_duplicates(subset = ['video_id','meeting_date'])[['video_id','meeting_date']]
dates['meeting_date'] = pd.to_datetime(dates['meeting_date'], format='%Y-%m-%d', errors='coerce')

deepseek_df = pd.read_csv(init_deepseek_results)
df = pd.merge(deepseek_df,dates, on = 'video_id', how = 'left')



In [5]:
# remove NaT and non recent data, filter to dates of interest (year 2024 & 2025)
df_24_25 = df[df['meeting_date'].astype(str).str.startswith("2024") | df['meeting_date'].astype(str).str.startswith("2025")]

# save updated file
df_24_25.to_csv(DEEPSEEK_RESULTS) # 2972 rows aka videos


In [7]:
# run gpt annotation process for all sections in the list of files 
run_gpt_annotation_for_file(files = [DEEPSEEK_RESULTS]) 

#Note: here I'm only doing one file, but could run across multiple files in the future


/Users/keeganveazey/Desktop/PCG_school_closures/Most recent run - 2024:25 original
1  /  1
Index(['Unnamed: 0.1', 'Unnamed: 0', 'centroid_lat', 'centroid_lon', 'leaid',
       'place_name', 'state_name', 'video_id', 'caption_sentences',
       'caption_sentence_topics', 'initial_keywords', 'kw_text_chunks',
       'relevancy_category', 'indices', 'kept_sections', 'meeting_date'],
      dtype='object')
total number of transcript sections to be annotated 310
Number of unique video ids in this file 2971
File saved at: PROBLEM_SOLUTION_GPT4o_JSON_RESULTS/GPT4o_output_deepseek_results_24_25_only.csv.json
[
  [],
  [
    {
      "video_id": "jV5xKMXARjQ",
      "problem_solution_dictionary": {
        "Displacement and instability for students, educators, and staff following school closure or crisis (such as fire)": [
          "Prioritize continuity and routine for students to minimize educational and emotional disruption",
          "Support, preserve, and protect signature school programs